In [2]:
#
# Proj learning
#
#
import numpy as np
import matplotlib.pyplot as plt
from pyproj import Proj
import pyproj

<hr>
Modified    BY  Reason <br> 
22-Jan-22   CBL Original <br>
<hr>
References: <br>
 https://en.wikipedia.org/wiki/Geographic_coordinate_conversion#From_geodetic_to_ECEF_coordinates 
 <br>
https://pyproj4.github.io/pyproj/stable/
<br>
https://pyproj4.github.io/pyproj/stable/examples.html
<br>
https://proj.org/faq.html#what-is-the-best-format-for-describing-coordinate-reference-systems
<hr>
Notes: 22-Jan-22, Ok my version of Proj is 4.9.3 15-Aug-2016, The latestest release is 8.2.1. Not sure how I get this fixed on my linux box.<br>
Ok the bottom line is that on Ubuntu, only proj4 is supported. <br> 
Look at this page: https://proj.org/install.html <br> 
It is recommending using conda to install the latest version. 

In [27]:
print(dir(pyproj))

['Geod', 'Proj', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', '__version__', '_convertback', '_copytobuffer', '_copytobuffer_return_scalar', '_dict2string', '_proj', 'array', 'datadir', 'math', 'os', 'pj_ellps', 'pj_list', 'pyproj_datadir', 'set_datapath', 'string_types', 'sys', 'test', 'transform']


In [3]:
# Calculate UTM zone
Latitude  = 41.5
Longitude = -71.316
Zone = Longitude + 180.0 # in degrees
Zone = np.ceil(Zone/6.0)
print ("Zone=",Zone)
#
# https://coordinates-converter.com/en/decimal/41.499976,-71.315992?karte=OpenStreetMap&zoom=8
#
# Lat = 41.5, Lon = -71.316
#
# Zone: 19
# False Easting: 306694.305
# False Northing: 4596851.081

Zone= 19.0


In [21]:
#pyproj.show_versions()
#Proj.Print()
print(dir(Proj))

['__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_fwd', '_inv', 'is_geocent', 'is_latlong', 'proj_version', 'srs', 'to_latlong']


In [25]:
p = Proj(proj='utm',zone=Zone,ellps='WGS84') # use kwargs
x,y = p(Longitude, Latitude)
'x=%9.3f y=%11.3f' % (x,y)

'x=306693.709 y=4596853.764'

In [6]:
'lon=%8.3f lat=%5.3f' % p(x,y,inverse=True)

'lon= -71.316 lat=41.500'

In [7]:
dr = np.sqrt(np.power(x-306694.305,2.0)+np.power(y-4596851.081,2.0))
print("Difference: ",dr," m")

Difference:  2.748105193722538  m


This is self consistent, but differs from the online tool. 

In [26]:
# Not sure how this is done
print(p.proj_version)

4.93


In [9]:
# create another projection, ECEF
#ecef = Proj(proj='geocent',zone=10,ellps='WGS84')
ecef = Proj(proj='geocent', ellps='WGS84', datum='WGS84')
lla = Proj(proj='latlong', ellps='WGS84', datum='WGS84')

In [10]:
# use the web example first. 
# https://stackoverflow.com/questions/30307311/python-pyproj-convert-ecef-to-lla
# Example position data, should be somewhere in Germany
x = 652954.1006
y = 4774619.7919
z = -2217647.7937
#
ecef = Proj(proj='geocent', ellps='WGS84', datum='WGS84')
lla = Proj(proj='latlong', ellps='WGS84', datum='WGS84')
lat, lon, alt = pyproj.transform(ecef, lla, x, y, z, radians=True)
# 
# Lat and lon are reversed in the original example if this is supposed to be Germany
# but this still ends up in Greenland. Z is pretty wonky. 
#
print(np.rad2deg(lat)," ",np.rad2deg(lon)," ", alt)

82.2128095673836   -24.887220777850917   -1069542.1723240195


In [11]:
# Try this with a different setup of XYZ based in RI. 
# http://www.sysense.com/products/ecef_lla_converter/index.html
# Input 41.5, -71.316, 32.0
# output: 1532547.72, -4531884.04, 4204193.46
x = 1532547.72
y = -4531884.04
z = 4204193.46
lat, lon, alt = pyproj.transform(ecef, lla, x, y, z, radians=True)
print(np.rad2deg(lat)," ",np.rad2deg(lon)," ", alt)

-71.31599997794198   41.499999997514784   31.99699786771089


This appears to be working ok. 
How about range and bearing?

In [12]:
# First, this is said to be the newer transformer protocol. 
# Transformer is in version 3.3 and greater of pyproj. 
#transformer = pyproj.Transformer.from_crs(
#    {"proj":'geocent', "ellps":'WGS84', "datum":'WGS84'},
#    {"proj":'latlong', "ellps":'WGS84', "datum":'WGS84'},
#    )
#lon1, lat1, alt1 = transformer.transform(x,y,z,radians=False)
#print (lat1, lon1, alt1 )